In [71]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [8]:
# Commute Mode: Table B08006
# Workers 16 years and over
mode_variables = {
    'B08006_001E': 'total',
    'B08006_001M': 'total_moe',
    'B08006_002E': 'vehicle',
    'B08006_002M': 'vehicle_moe',
    'B08006_009E': 'bus',
    'B08006_009M': 'bus_moe',
    'B08006_010E': 'subway',
    'B08006_010M': 'subway_moe',
    'B08006_011E': 'rail',
    'B08006_011M': 'rail_moe',
    'B08006_012E': 'lightrail',
    'B08006_012M': 'lightrail_moe',
    'B08006_013E': 'ferry',
    'B08006_013M': 'ferry_moe',
    'B08006_014E': 'bike',
    'B08006_014M': 'bike_moe',
    'B08006_015E': 'walk',
    'B08006_015M': 'walk_moe',
    'B08006_016E': 'vehicle_other',
    'B08006_016M': 'vehicle_other_moe',
    'B08006_017E': 'wfh',
    'B08006_017M': 'wfh_moe'
}
mode_columns_out = [
    'total', 'total_moe',
    'pct_rail_all', 'pct_rail_all_moe', 
    'pct_other', 'pct_other_moe',
    'pct_vehicle', 'pct_vehicle_moe',
    'pct_bus', 'pct_bus_moe',
    'pct_wfh','pct_wfh_moe',
    'pct_walk_or_bike', 'pct_walk_or_bike_moe'
]

In [3]:
def combine_modes(in_df):
    '''
    Outputs: simplified mode breakdown with MOEs
    '''
    df = in_df.copy()
    
    ### AGGREGATE ESTIMATES
    # Define a list of columns to combine into "other"
    other_cols = ['ferry', 'vehicle_other']
    df['other'] = df[other_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    other_moes = [f'{col}_moe' for col in other_cols]
    df['other_moe'] = (df[other_moes]**2).sum(axis='columns')**0.5

    # Rail aggregate
    rail_cols = ['subway', 'rail', 'lightrail']
    df['rail_all'] = df[rail_cols].sum(axis='columns')
    rail_moes = [f'{col}_moe' for col in rail_cols]
    df['rail_all_moe'] = (df[rail_moes]**2).sum(axis='columns')**0.5

    # Active aggregate
    active_cols = ['walk', 'bike']
    df['walk_or_bike'] = df[active_cols].sum(axis='columns')
    active_moes = [f'{col}_moe' for col in active_cols]
    df['walk_or_bike_moe'] = (df[active_moes]**2).sum(axis='columns')**0.5

    for group in ['rail_all', 'other', 'vehicle', 'bus', 'walk_or_bike', 'wfh']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df['total'] < 25, f'pct_{group}'] = float('NaN')
        df.loc[df['total'] < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')
    
    return df

In [4]:
def combine_places(place_1, place_2):
    
    df_1 = place_1.copy()
    df_2 = place_2.copy()
    df = pd.DataFrame()
    
    #Total the totals
    df["total"] = df_1["total"]+df_2["total"]
    df["total_moe"] = np.sqrt(df_1['total_moe']**2 + df_2['total_moe']**2)
    
    ### CALCULATE PROPORTIONS
    for group in ['rail_all', 'other', 'vehicle', 'bus', 'walk_or_bike', 'wfh']:

        df[group] = df_1[group] + df_2[group]
        df[f'{group}_moe'] = np.sqrt(df_1[f'{group}_moe']**2 + df_2[f'{group}_moe']**2)
        
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df.total < 25, f'pct_{group}'] = float('NaN')
        df.loc[df.total < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')

    return df

In [15]:
def get_place_precombo(year_in, place_type, place_num):
    # Call the census API for a place before it is combined with another place
    
    df = pd.DataFrame(
        c.acs5.get(
            list(mode_variables.keys()),
            {'for': place_type+':'+place_num, 'in': 'state:06'},
            year=year_in
        )
    )

    df = df.rename(columns=mode_variables)
    df = combine_modes(df)
    
    return df

In [24]:
def get_county_df(year_in):
    # County
    df = pd.DataFrame(
        c.acs5.get(
            list(mode_variables.keys()),
            {'for': 'county:013', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=mode_variables)
    df_out = combine_modes(df)
    df_out = df_out[mode_columns_out]
    df_out.insert(0, "year", year_in)
    df_out.insert(0, "NAME", "Contra Costa County")

    return df_out

## County

In [25]:
# for 5yr ACS 2017 and 2022
# Get ACS Table B08006 in Contra Costa County
df_county_2017 = get_county_df(2017)
df_county_2022 = get_county_df(2022)

In [26]:
df_out_county = pd.concat([df_county_2017, df_county_2022])
df_out_county

,NAME,year,total,total_moe,pct_rail_all,pct_rail_all_moe,pct_other,pct_other_moe,pct_vehicle,pct_vehicle_moe,pct_bus,pct_bus_moe,pct_wfh,pct_wfh_moe,pct_walk_or_bike,pct_walk_or_bike_moe
0,Contra Costa County,2017,520162.0,3157.0,0.085944,0.003425,0.014538,0.001159,0.797853,0.003129,0.017137,0.001272,0.062655,0.002408,0.021872,0.001842
0,Contra Costa County,2022,556391.0,2772.0,0.059550,0.003359,0.016350,0.001289,0.717609,0.006148,0.015268,0.001726,0.169755,0.005156,0.021469,0.001772


## Cities

In [16]:
# 2017
# Richmond City
df_r_2017 = get_place_precombo(2017, 'place', '60620')
# North Richmond
df_nr_2017 = get_place_precombo(2017, 'place', '52162')

# 2022
# Richmond City
df_r_2022 = get_place_precombo(2022, 'place', '60620')
# North Richmond
df_nr_2022 = get_place_precombo(2022, 'place', '52162')

In [17]:
#combo Richmond with North Richmond
# 2017
df_richmond_2017 = combine_places(df_r_2017, df_nr_2017)
df_richmond_2017 = df_richmond_2017[mode_columns_out]
df_richmond_2017.insert(0, "NAME", "Richmond")
df_richmond_2017.insert(0, "year", 2017)

# 2022
df_richmond_2022 = combine_places(df_r_2022, df_nr_2022)
df_richmond_2022 = df_richmond_2022[mode_columns_out]
df_richmond_2022.insert(0, "NAME", "Richmond")
df_richmond_2022.insert(0, "year", 2022)

df_out_richmond = pd.concat([df_richmond_2017, df_richmond_2022])

In [18]:
df_out_richmond

,year,NAME,total,total_moe,pct_rail_all,pct_rail_all_moe,pct_other,pct_other_moe,pct_vehicle,pct_vehicle_moe,pct_bus,pct_bus_moe,pct_wfh,pct_wfh_moe,pct_walk_or_bike,pct_walk_or_bike_moe
0,2017,Richmond,51950.0,1215.375662,0.104812,0.010790,0.015188,0.004112,0.779134,0.012159,0.033051,0.006155,0.043677,0.006499,0.024139,0.007090
0,2022,Richmond,57615.0,1505.074749,0.058700,0.010614,0.023709,0.006370,0.751176,0.018209,0.033672,0.008699,0.112367,0.012581,0.020377,0.005034


In [19]:
# Pittsburg and Bay Point
# 2017
# Pittsburg
df_p_2017 = get_place_precombo(2017, 'place', '57456')
# Bay Point
df_bp_2017 = get_place_precombo(2017, 'place', '04415')

# 2022
# Pittsburg
df_p_2022 = get_place_precombo(2022, 'place', '57456')
# Bay Point
df_bp_2022 = get_place_precombo(2022, 'place', '04415')

In [20]:
#combo Pittsburg and Bay Point
# 2017
df_pbp_2017 = combine_places(df_p_2017, df_bp_2017)
df_pbp_2017 = df_pbp_2017[mode_columns_out]
df_pbp_2017.insert(0, "NAME", "Pittsburg/Bay Point")
df_pbp_2017.insert(0, "year", 2017)

# 2022
df_pbp_2022 = combine_places(df_p_2022, df_bp_2022)
df_pbp_2022 = df_pbp_2022[mode_columns_out]
df_pbp_2022.insert(0, "NAME", "Pittsburg/Bay Point")
df_pbp_2022.insert(0, "year", 2022)

df_out_pbp = pd.concat([df_pbp_2017, df_pbp_2022])

In [21]:
# El Cerrito
df_ec_2017 = get_place_precombo(2017, 'place', '21796')
df_ec_2017 = df_ec_2017[mode_columns_out]
df_ec_2017.insert(0, "NAME", "El Cerrito")
df_ec_2017.insert(0, "year", 2017)
df_ec_2022 = get_place_precombo(2022, 'place', '21796')
df_ec_2022 = df_ec_2022[mode_columns_out]
df_ec_2022.insert(0, "NAME", "El Cerrito")
df_ec_2022.insert(0, "year", 2022)

df_out_ec = pd.concat([df_ec_2017, df_ec_2022])

# Lafayette
df_la_2017 = get_place_precombo(2017, 'place', '39122')
df_la_2017 = df_la_2017[mode_columns_out]
df_la_2017.insert(0, "NAME", "Lafayette")
df_la_2017.insert(0, "year", 2017)
df_la_2022 = get_place_precombo(2022, 'place', '39122')
df_la_2022 = df_la_2022[mode_columns_out]
df_la_2022.insert(0, "NAME", "Lafayette")
df_la_2022.insert(0, "year", 2022)

df_out_la = pd.concat([df_la_2017, df_la_2022])

In [27]:
df_out = pd.concat([df_out_county,df_out_la,df_out_ec,df_out_pbp,df_out_richmond])

In [29]:
df_out.to_csv('vehicle/modes.csv', index=False)

In [28]:
df_out

,NAME,year,total,total_moe,pct_rail_all,pct_rail_all_moe,pct_other,pct_other_moe,pct_vehicle,pct_vehicle_moe,pct_bus,pct_bus_moe,pct_wfh,pct_wfh_moe,pct_walk_or_bike,pct_walk_or_bike_moe
0,Contra Costa County,2017,520162.0,3157.000000,0.085944,0.003425,0.014538,0.001159,0.797853,0.003129,0.017137,0.001272,0.062655,0.002408,0.021872,0.001842
0,Contra Costa County,2022,556391.0,2772.000000,0.059550,0.003359,0.016350,0.001289,0.717609,0.006148,0.015268,0.001726,0.169755,0.005156,0.021469,0.001772
0,Lafayette,2017,11738.0,533.000000,0.193559,0.024083,NaN,NaN,0.657182,0.030158,NaN,NaN,0.119697,0.017045,NaN,NaN
0,Lafayette,2022,11709.0,566.000000,0.092066,0.018208,NaN,NaN,0.502007,0.046196,NaN,NaN,0.375609,0.052279,NaN,NaN
0,El Cerrito,2017,12281.0,451.000000,0.208534,0.017753,NaN,NaN,0.597508,0.022617,0.043563,0.010629,0.106343,0.015054,0.033059,0.009713
0,El Cerrito,2022,13356.0,516.000000,0.156110,0.021859,NaN,NaN,0.523510,0.029526,0.018643,0.006851,0.239368,0.025159,0.047020,0.011360
0,Pittsburg/Bay Point,2017,40608.0,1226.923388,0.081117,0.011105,0.016105,0.004411,0.846853,0.015565,0.015391,0.003807,0.024823,0.005191,0.015711,0.005026
0,Pittsburg/Bay Point,2022,46663.0,1406.475382,0.057433,0.010504,NaN,NaN,0.843859,0.013864,0.010158,0.003542,0.063584,0.008233,NaN,NaN
0,Richmond,2017,51950.0,1215.375662,0.104812,0.010790,0.015188,0.004112,0.779134,0.012159,0.033051,0.006155,0.043677,0.006499,0.024139,0.007090
0,Richmond,2022,57615.0,1505.074749,0.058700,0.010614,0.023709,0.006370,0.751176,0.018209,0.033672,0.008699,0.112367,0.012581,0.020377,0.005034


## Charts

In [36]:
# data prep
dfc = df_out.copy()
dfc.reset_index(drop=True, inplace=True)
chart_cols = ["NAME", "year", "pct_rail_all","pct_rail_all_moe","pct_bus", "pct_bus_moe"]
dfc = dfc[chart_cols]
dfc.drop([2,3,4,5], inplace=True)

In [37]:
dfc

,NAME,year,pct_rail_all,pct_rail_all_moe,pct_bus,pct_bus_moe
0,Contra Costa County,2017,0.085944,0.003425,0.017137,0.001272
1,Contra Costa County,2022,0.059550,0.003359,0.015268,0.001726
6,Pittsburg/Bay Point,2017,0.081117,0.011105,0.015391,0.003807
7,Pittsburg/Bay Point,2022,0.057433,0.010504,0.010158,0.003542
8,Richmond,2017,0.104812,0.010790,0.033051,0.006155
9,Richmond,2022,0.058700,0.010614,0.033672,0.008699


In [85]:
# Rail
rail_chart = alt.Chart(dfc).mark_bar(size=35).encode(
    x=alt.X('year:N').title('').axis(labelAngle=0),
    y=alt.Y('pct_rail_all:Q').scale(domain=(0,.15)).axis(format='%').title('Percent of All Commute Modes'),
    color=alt.Color('year:N').title('Year'),
    #order = alt.Order('color_financial_recovery_sort_index:Q'),
).properties(width=100)

rail_errors = alt.Chart().mark_errorbar().encode(
    x='year:N',
    y=alt.Y('pct_rail_all:Q').scale(zero=False).title(''),
    yError='pct_rail_all_moe:Q'
)

alt.layer(rail_chart, rail_errors, data=dfc).facet(
    column=alt.Column("NAME:N", header=alt.Header(labelOrient='bottom')).title('')
)

alt.FacetChart(...)

In [86]:
# Bus
bus_chart = alt.Chart(dfc).mark_bar(size=35).encode(
    x=alt.X('year:N', title=None).axis(labelAngle=0),
    y=alt.Y('pct_bus:Q').scale(domain=(0,.15)).axis(format='%').title('Percent of All Commute Modes'),
    color=alt.Color('year:N').title('Year'),
    text=alt.Text('pct_bus:Q', format='.1%')
    #order = alt.Order('color_financial_recovery_sort_index:Q'),
).properties(width=100)

bus_errors = alt.Chart().mark_errorbar().encode(
    x='year:N',
    y=alt.Y('pct_bus:Q').scale(zero=False).title(''),
    yError='pct_bus_moe:Q'
)

alt.layer(bus_chart, bus_errors, data=dfc).facet(
    column=alt.Column("NAME:N", header=alt.Header(labelOrient='bottom')).title('')
)

alt.FacetChart(...)